In [29]:
# Set the root directory accordingly
ROOT_DIR = "d:/Coding/MH4518"

import os, sys
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)

In [27]:
import numpy as np
import pandas as pd

### Loading Data

Product: https://derivative.credit-suisse.com/ch/ch/en/detail/drop-back-certificate-s-p-500/CH1199361879/119936187
<br>
<li> Start date: 09-08-2022
<li> End date: 09-11-2023
<br>

TODO: Update datasets (currently only till Oct 20)

In [33]:
asset_prices = pd.read_csv("./data/asset_price_1_year.csv")
product_prices = pd.read_csv("./data/product_price_1_year.csv")

### Payoff Function

In [5]:
def find_trigger_price(S_path, trigger):
    '''
    Returns the price that triggered the event, if it exists, else None.
    '''
    # Return the first price that triggers the event
    for price in S_path:
        if price < trigger:
            return price
        
    return None


def calculate_payoff(S_path, S_0, Y_0 = 1000, rate = 0.0985, years = 3):
    '''
    Computes the payoff of a simulated path.
    '''
    S_T = S_path[-1]
    S_t1 = find_trigger_price(S_path, trigger = 0.90*S_0)
    S_t2 = find_trigger_price(S_path, trigger = 0.85*S_0)
    S_t3 = find_trigger_price(S_path, trigger = 0.80*S_0)

    # 1. Fixed returns independent of trigger events
    Y_T = (0.55*Y_0) * (S_T/S_0)    

    # 2. Adjusted returns after trigger events
    if S_t3 is not None:
        Y_T += (0.15*Y_0) * (S_T/S_t1 + S_T/S_t2 + S_T/S_t3)

    elif S_t2 is not None:
        Y_T += (0.15*Y_0) * (S_T/S_t1 + S_T/S_t2)

    elif S_t1 is not None:
        Y_T += (0.15*Y_0) * (S_T/S_t1)

    # 3. Daily accrued interest of 9.85% p.a.
    if S_t3 is not None:
        principal = 0.0

    elif S_t2 is not None:
        principal = (0.15*Y_0)

    elif S_t1 is not None:
        principal = (0.30*Y_0)

    else:
        principal = (0.45*Y_0)

    Y_T += principal * rate * years
    return Y_T

In [36]:
# Testing the payoff function
S_path_partial = [3790.38, 3800.15, 3373.44, 4000.00, 3183.92, 2994.40]
S_0 = 3790.38
S_T = [0.0, 2653.27, 3032.3, 3411.34, 3790.38, 4169.42, 4548.46, 4927.49]

for final_closing_price in S_T:
    S_path = S_path_partial + [final_closing_price]
    payoff = calculate_payoff(S_path, S_0)

    print(f"S_T = {final_closing_price} \t   Performance of Asset = {100*(final_closing_price - S_0)/S_0:.2f}% \t Performance of Certificate = {(payoff - 1000)/10:.2f}% \t Payoff = {payoff:.2f}")

S_T = 0.0 	   Performance of Asset = -100.00% 	 Performance of Certificate = -100.00% 	 Payoff = 0.00
S_T = 2653.27 	   Performance of Asset = -30.00% 	 Performance of Certificate = -23.91% 	 Payoff = 760.89
S_T = 3032.3 	   Performance of Asset = -20.00% 	 Performance of Certificate = -13.04% 	 Payoff = 869.59
S_T = 3411.34 	   Performance of Asset = -10.00% 	 Performance of Certificate = -2.17% 	 Payoff = 978.29
S_T = 3790.38 	   Performance of Asset = 0.00% 	 Performance of Certificate = 8.70% 	 Payoff = 1086.98
S_T = 4169.42 	   Performance of Asset = 10.00% 	 Performance of Certificate = 19.57% 	 Payoff = 1195.68
S_T = 4548.46 	   Performance of Asset = 20.00% 	 Performance of Certificate = 30.44% 	 Payoff = 1304.38
S_T = 4927.49 	   Performance of Asset = 30.00% 	 Performance of Certificate = 41.31% 	 Payoff = 1413.08
